In [ ]:
# Install the required libraries
!pip install transformers
!pip install torch
!pip install nltk

In [ ]:
# Import necessary libraries
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
# Import necessary libraries
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
# Download NLTK resources
nltk.download("stopwords")
nltk.download("punkt")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# # Load pre-trained BERT model and tokenizer
# model_name = "bert-base-uncased"
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name)

# Load pre-trained BART model and tokenizer for summarization
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
# Define preprocessing functions
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return " ".join(filtered_text)

In [ ]:
# Define a function for abstract summarization with preprocessing
def generate_summary(text):
    # Preprocess the input text (you can customize this based on your specific needs)
    # text = text.lower()  # Convert to lowercase
    # text = remove_stopwords(text)

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate summary using BART for summarization
    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
# Example usage
input_text = """To do that, stick to sturdy classic fonts like serifs and sans-serifs. Serifs, like Times New Roman, give off a very classy feel, and sans-serifs, like Arial, give off that professional vibe while being a bit more modern."""
result_summary = generate_summary(input_text)
print("Original Text:\n", input_text)
print("\nBERT Generated Summary:\n", result_summary)

Original Text:
 To do that, stick to sturdy classic fonts like serifs and sans-serifs. Serifs, like Times New Roman, give off a very classy feel, and sans-serifs, like Arial, give off that professional vibe while being a bit more modern.

BERT Generated Summary:
 To do that, stick to sturdy classic fonts like serifs and sans-serifs. Serifs, like Times New Roman, give off a very classy feel. Sans-Serifs give off that professional vibe while being a bit more modern.


In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=25b455b95a4bba1fe5ef3221d9d5b426ec7a3449c4b4bd83a6abe07750fac243
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

In [ ]:
!pip install rouge


In [ ]:
from rouge import Rouge

# Define the generated summary and the reference summary
generated_summary = result_summary
reference_summary = input_text
# Initialize the ROUGE object
rouge = Rouge()
# Calculate ROUGE for the generated and reference summaries
scores = rouge.get_scores(generated_summary, reference_summary)
# Print the results
print(scores)

[{'rouge-1': {'r': 0.90625, 'p': 0.9354838709677419, 'f': 0.9206349156361804}, 'rouge-2': {'r': 0.8333333333333334, 'p': 0.9090909090909091, 'f': 0.8695652124007562}, 'rouge-l': {'r': 0.90625, 'p': 0.9354838709677419, 'f': 0.9206349156361804}}]


In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
from bert_score import score

In [ ]:
# from bert_score import score

hypotheses = [result_summary]
references = [input_text]
P, R, F1 = score(hypotheses, references, model_type="bert-base-uncased")  # Specify the BERT model type

print("Precision:", P.mean().item())
print("Recall:", R.mean().item())
print("F1 Score:", F1.mean().item())


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Precision: 0.9720218181610107
Recall: 0.9423763751983643
F1 Score: 0.9569695591926575


In [ ]:
!pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=ed0380129a54f2bf73fd7a1b6625b8d4093e9ee4081ab06831f59575480f5ec0
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [ ]:
from googletrans import Translator
from transformers import pipeline

def translate_text(text, source_lang='en', target_lang='te'):
    translator = Translator()
    translation = translator.translate(text, src=source_lang, dest=target_lang)
    return translation.text
telugu_answer = translate_text(result_summary)
print(telugu_answer)

అలా చేయడానికి, సెరిఫ్‌లు మరియు సాన్స్-సెరిఫ్‌లు వంటి ధృ dy నిర్మాణంగల క్లాసిక్ ఫాంట్‌లకు కట్టుబడి ఉండండి.సెరిఫ్‌లు, టైమ్స్ న్యూ రోమన్ లాగా, చాలా క్లాస్సి అనుభూతిని ఇస్తాయి.సాన్స్-సెరిఫ్‌లు కొంచెం ఆధునికంగా ఉన్నప్పుడు ఆ ప్రొఫెషనల్ వైబ్‌ను ఇస్తాయి.


In [ ]:
from googletrans import Translator
from transformers import pipeline

def translate_text(text, source_lang='en', target_lang='Hi'):
    translator = Translator()
    translation = translator.translate(text, src=source_lang, dest=target_lang)
    return translation.text
telugu_answer = translate_text(result_summary)
print(telugu_answer)

ऐसा करने के लिए, सेरिफ़ और सैंस-सेरिफ़ जैसे मजबूत क्लासिक फोंट से चिपके रहें।सेरिफ़, टाइम्स न्यू रोमन की तरह, एक बहुत ही उत्तम दर्जे का एहसास देते हैं।Sans-Serifs थोड़ा अधिक आधुनिक होने के दौरान उस पेशेवर खिंचाव को छोड़ देते हैं।


In [ ]:
from googletrans import Translator
from transformers import pipeline

def translate_text(text, source_lang='en', target_lang='ta'):
    translator = Translator()
    translation = translator.translate(text, src=source_lang, dest=target_lang)
    return translation.text
telugu_answer = translate_text(result_summary)
print(telugu_answer)

அதைச் செய்ய, செரிஃப்ஸ் மற்றும் சான்ஸ்-செரிஃப்ஸ் போன்ற துணிவுமிக்க கிளாசிக் எழுத்துருக்களுடன் ஒட்டிக்கொள்க.டைம்ஸ் நியூ ரோமன் போன்ற செரிஃப்கள், மிகவும் கம்பீரமான உணர்வைத் தருகின்றன.சான்ஸ்-செரிஃப்ஸ் அந்த தொழில்முறை அதிர்வை இன்னும் கொஞ்சம் நவீனமாகக் கொடுக்கிறார்.


In [ ]:
from googletrans import Translator
from transformers import pipeline

def translate_text(text, source_lang='en', target_lang='ka'):
    translator = Translator()
    translation = translator.translate(text, src=source_lang, dest=target_lang)
    return translation.text
telugu_answer = translate_text(result_summary)
print(telugu_answer)

ამის გასაკეთებლად, მიჰყევით მკაცრ კლასიკურ შრიფტებს, როგორიცაა Serifs და Sans-serifs.სერია, ისევე როგორც Times New Roman, ძალიან კლასიკური შეგრძნება გამოთქვამს.Sans-serifs აჩუქებს ამ პროფესიონალურ ხედვას, ხოლო ცოტა უფრო თანამედროვეა.
